# LinkedIn AI Agent System Documentation

## Agent Overview

This LinkedIn AI Agent is a comprehensive automated system designed to streamline LinkedIn content creation and publishing. The agent combines multiple AI-powered capabilities to create a complete workflow: it scrapes trending topics from LinkedIn, analyzes successful post examples, generates contextually relevant content, and automatically publishes posts while maintaining consistent brand voice and messaging.

### Key Features:
- **Trend Analysis**: Automatically discovers trending AI and technology topics on LinkedIn
- **Content Intelligence**: Uses vector database of successful post examples to maintain consistent tone and style
- **Automated Publishing**: Directly posts generated content to LinkedIn
- **Calendar Integration**: Creates reminders for scheduled posting activities

[VIDEO TUTORIAL](https://drive.google.com/file/d/1OmsxxzVPXHD5mqDOxhr1ktXy4l1qxvFW/view?usp=drive_link)

## Code Documentation

### 1. Environment Setup and API Configuration

Installs the aiXplain SDK and configures API authentication. Sets up necessary imports for agent creation, task management, and tool integration.

In [ ]:
# Initialize aiXplain SDK environment
!pip -q install aixplain

import os
os.environ["AIXPLAIN_API_KEY"] = "<YOUR_AIXPLAIN_API_KEY>"

from aixplain.factories import AgentFactory, TeamAgentFactory, ModelFactory, ToolFactory, IndexFactory
from aixplain.modules.agent.agent_task import AgentTask
from aixplain.modules.model.record import Record

### 2. LinkedIn Tool Configuration
Retrieves the LinkedIn integration tool from aiXplain marketplace and configures it with specific actions for posting content and retrieving user information. The action scope limits the tool to only the required LinkedIn operations.

[Learn how to use Integrations in aiXplain →](https://docs.aixplain.com/concepts/tools/integrations)

In [ ]:
linkedIn_tool = ToolFactory.get("<YOUR_LINKEDIN_COMPOSIO_INTEGRATION_ID>") # Your LinkedIn tool from your dashboard

print("LinkedIn Tool Actions:", [action.code for action in linkedIn_tool.actions]) # List all LinkedIn actions

# Select the tool actions you want to add to your agent
needed_actions = {
    "LINKEDIN_CREATE_LINKED_IN_POST",
    "LINKEDIN_GET_MY_INFO",
}

# Add the selected actions to your tool's scope
linkedIn_tool.action_scope = [a for a in linkedIn_tool.actions if a.code in needed_actions]

# len(linkedIn_tool.action_scope), [a.code for a in linkedIn_tool.action_scope]


LinkedIn Tool Actions: ['LINKEDIN_CREATE_LINKED_IN_POST', 'LINKEDIN_DELETE_LINKED_IN_POST', 'LINKEDIN_GET_COMPANY_INFO', 'LINKEDIN_GET_MY_INFO']


### 3. LinkedIn Tool Testing and Validation

Tests the LinkedIn tool functionality by examining available actions and their required parameters. Performs validation runs to ensure the tool can retrieve user information and create posts successfully.

In [ ]:
# Print a scoped action
action = [action for action in linkedIn_tool.actions if action.code == "LINKEDIN_GET_MY_INFO"][0] # Action code and name

print(action)                           # Action name
linkedIn_tool.get_action_inputs(action) # Action parameters

Action(code=LINKEDIN_GET_MY_INFO, name=Get my info)


{}

In [ ]:
# Try an action
linkedIn_tool.run("LINKEDIN_GET_MY_INFO", {})

# This code gives you your LinkedIn profile ID needed for testing 'author_id': 'urn:li:person:<ID>'

ModelResponse(status=SUCCESS, data='{'response_dict': {'author_id': 'urn:li:person:lRQvNwgIyI', 'email': 'nuraahamdan@gmail.com', 'email_verified': True, 'family_name': 'Hamdan', 'given_name': 'Nur', 'locale': {'country': 'US', 'language': 'en'}, 'name': 'Nur Hamdan', 'picture': 'https://media.licdn.com/dms/image/v2/D5603AQF9WSzwahrJ8Q/profile-displayphoto-shrink_100_100/profile-displayphoto-shrink_100_100/0/1677792388427?e=1761177600&v=beta&t=kKZW0MHgExbGq1WFNHyD3SmfHT0tliC1XqX06Kn3M7o', 'sub': 'lRQvNwgIyI'}}', completed=True, run_time=0.463)

In [ ]:
# Print a scoped action
action = [action for action in linkedIn_tool.actions if action.code == "LINKEDIN_CREATE_LINKED_IN_POST"][0] # Action code and name

print(action)                           # Action name
linkedIn_tool.get_action_inputs(action) # Action parameters

Action(code=LINKEDIN_CREATE_LINKED_IN_POST, name=Create a LinkedIn post)


{'author': {'name': 'Author',
  'code': 'author',
  'value': [],
  'availableOptions': [],
  'datatype': 'string',
  'allowMulti': False,
  'supportsVariables': False,
  'defaultValue': [],
  'required': True,
  'fixed': False,
  'description': "The URN of the LinkedIn member or organization creating the post. Use the GET_USER_INFO action to retrieve the 'author_id' (URN) for the authenticated user. For an organization, the URN will be in the format 'urn:li:organization:{id}'."},
 'commentary': {'name': 'Commentary',
  'code': 'commentary',
  'value': [],
  'availableOptions': [],
  'datatype': 'string',
  'allowMulti': False,
  'supportsVariables': False,
  'defaultValue': [],
  'required': True,
  'fixed': False,
  'description': 'The main text content of the post. This field supports plain text and @-mentions (e.g., @[LinkedIn Member](urn:li:person:xxxx)).'},
 'distribution': {'name': 'Distribution',
  'code': 'distribution',
  'value': [],
  'availableOptions': [],
  'datatype': 'o

In [ ]:
# Try an action - Copy your profile ID from the previous print
linkedIn_tool.run("LINKEDIN_CREATE_LINKED_IN_POST", {'author':'urn:li:person:<ID>', 'commentary':"Demo"}) # BE CREFUL - This will post a LinkedIn post with the word "Demo" in your profile

ModelResponse(status=SUCCESS, data='{'response_data': {'content': '', 'reason': 'Created', 'share_id': 'urn:li:share:7373865885951963136', 'status_code': 201}}', completed=True, run_time=0.628)

### 4. Web Search Tool Setup
Configures a web scraping tool (Tavily Search) to find trending content and posts. Domain restriction ensures searches are focused on relevant platforms (LinkedIn and YouTube) for better content quality and relevance.

In [ ]:
# Get a search tool from the marketplace
scrape_tool = ToolFactory.get("6736411cf127849667606689") # Tavily Search Tool

# Limit the domains the tool is allowed to search
scrape_tool.input_params['include_domains'] = ['linkedIn.com']

### 5. Post Examples Database Creation
Defines a collection of high-quality LinkedIn post examples that demonstrate the desired tone, style, and messaging approach. These posts serve as training data for maintaining consistent voice and format in generated content.

In [ ]:
data = [
    {
        "id": "1",
        "post": """You’ve heard of greenwashing. But what about Agent Washing? 🧼

It’s when a tool is marketed as an “AI agent” — but under the hood, it’s just a prompt follower.
🔸 No planning
🔸 No acting
🔸 No adaptation

Just linear LLM responses with a glossy UI.
And it’s everywhere.

Every week, a customer asks us:
“How are you different from X?”
We dig in. And more often than not…
It’s classic agent washing.

Agent washing doesn’t just confuse the market.
 It misleads buyers, causes deployment failures, and exposes enterprises to:
🔐 Governance gaps
⚠️ Security risks
🧩 Integration breakdowns

Here’s the truth: Most platforms today support assistants, chatbots, or copilots.
They generate. They respond.
But they don’t act autonomously.

True agents require a fundamentally different backbone:
➡️ An orchestration engine ⬅️

🧠 What Is an Orchestration Engine?
An orchestration engine is the core runtime system that enables agents to plan, act, and coordinate across tools, data, and other agents — all under enterprise governance. It manages the execution flow, enforces policy, controls memory, handles failure, and ensures observability.

To qualify as a production-grade agent orchestrator, a platform must provide:

✅ Agent coordination and delegation - Support for composing, routing, and synchronizing specialized agents across tasks
✅ Resilience mechanisms - Built-in support for retries, fallbacks, escalation, and adaptive replanning
✅ Memory and context management - Isolated, persistent memory per user/session/workflow to ensure safe, relevant execution
✅ Secure tool integration - Permissioned access to APIs and services with scoped credentials and runtime enforcement
✅ Governance and observability - End-to-end traceability of decisions, policy enforcement, audit trails, and structured logs
✅ Flexible deployment modes - Support for SaaS, VPC, on-premise, and hybrid environments to meet compliance and control needs

These dimensions form the basis of our buyer’s checklist 📃
Most vendors won’t meet even half this list.
Because they’re not building for agents — they’re building for prompts.

💡 At aiXplain, we built our orchestration engine to meet this checklist — and power real agents in production.""",
    },
    {
        "id": "2",
        "post": """
        Don’t assume LLM randomness is inevitable. That era is ending.

We’ve all seen it: same model, same input—different answer.
Even at temperature 0, where outputs should be deterministic.

For enterprises, that’s not just frustrating. It’s a dealbreaker.
🔐 Reproducibility isn’t academic—it’s foundational.
Without it, audits fail, automations break, and ROI becomes guesswork.

That’s why a quiet breakthrough from Thinking Machines matters:
📌 “Defeating Nondeterminism in LLM Inference”

The takeaway?
This isn’t a model problem. It’s an infrastructure flaw.
And it’s fixable.

Forward-looking teams are moving now—not when it’s safe, but while it’s strategic.
Because preparing for deterministic AI means more than just tweaking parameters.

It means rethinking infrastructure to support:
🔸 Trust-by-design
🔸 Scalable governance
🔸 Seamless agent deployment

🛠️ At aiXplain, we’re already building toward that future.
Running on enterprise-grade private infrastructure.
Partnering with integrators and innovators to deliver real-world, reproducible AI.

Infrastructure readiness is your moat.
Reproducibility is your foundation.
And the window to get ahead is now.

📄 Full article in the comments.

♻️ Repost if you’re not waiting for the baseline to shift—you’re preparing for it.
""",
    },
    {
        "id": "3",
        "post": """
        Why enterprise AI isn't just "software with a model." And why vibe-coding isn't catching up with AI engineers.

Traditional software is built on rules, determinism, and control.
You define inputs, write logic, run tests—and once deployed, behavior is repeatable and predictable.

But agentic AI systems don’t operate on fixed logic.
They reason, act, and adapt—in real time, with tools, models, and changing environments.

This shift breaks the traditional software contract. Here’s how:

🔁 Non-determinism - Same input, different output—depending on model context, retrievals, or tool states.
➕ Lets agents adapt responses to real-time context—enabling hyper-personalized decisions at scale.

🧠 Opaque reasoning chains - Decisions happen inside models, not code. Planning is dynamic, not traceable.
➕ Unlocks complex, emergent behavior—so systems can handle edge cases without manual rules.

🎛 Execution variability - Outcome depends on everything: tools, APIs, data freshness, model latency.
➕ Empowers agents to choose optimal tools on the fly—balancing speed, accuracy, and cost.

🧪 Continuous evaluation - Static unit tests won’t cut it. You need runtime checks for hallucination, groundedness, and cost.
➕ Flags hallucinations and cost spikes in real time—so issues are caught before users ever see them.

🔒 Governance and safety - In regulated sectors, systems must be policy-bound, observable, and fail-closed by design.
➕ Makes AI systems policy-compliant and audit-ready—turning risk into resilience.

🧩 Multi-agent orchestration - Logic spans agents that must collaborate, escalate, and self-correct.
➕ Enables scalable workflows—where agents collaborate, escalate, and self-correct like a real team.

This is a paradigm shift—from deterministic logic to probabilistic reasoning, from hard-coded flows to autonomous orchestration. Agentic AI systems require new engineering disciplines, not just clever prompts or vibe-coded wrappers.

To build enterprise-ready AI, you need:
 🔹 Infrastructure that understands agents
 🔹 Observability that captures reasoning
 🔹 Governance that enforces trust boundaries
 🔹 Deployment models that respect sovereignty
Until your architecture reflects the non-deterministic, tool-using, policy-bound nature of agents, you’re not building systems—you’re stacking guesses.

Enterprise AI isn’t about wrapping a model.
It’s about designing for behavior you don’t fully control—then making it safe, observable, and adaptive anyway.

That’s not a prompt.
That’s engineering.

♻️ Repost to help reframe how we talk about enterprise AI.
""",
    },
    {
        "id": "4",
        "post": """
        The biggest illusion in enterprise AI right now?
That agents = productivity.

Everyone’s launching agents: Customer support copilots. Document triagers. Workflow bots.

But peel back the demo—and you’ll often find:
🔸 No orchestration.
🔸 No shared memory.
🔸 No governance, observability, or auditability.
🔸 No support for real scale.
Just isolated tools with clever wrappers.

Here’s the hard truth:
If your agents don’t talk to each other...
If they can’t adapt, coordinate, or learn...
If you can’t govern or debug them at runtime...
👉 You don’t have an agentic system. You have automation theater 🎭

What we’re seeing now is a category shift. From tooling for models → to operating systems for agents.

That means rethinking everything:
1️⃣ From pipelines to policies - Can your system enforce privacy, RBAC, audit trails—across agents and models?

2️⃣ From scripts to orchestration - Can your agents route tasks, decompose goals, handle fallback—and evolve over time?

3️⃣ From POCs to platforms - Are you building agent-by-agent?
 Or creating infrastructure that supports 100+ agents—compliantly and observably?

The reframe:
Enterprises don’t just need more AI. They need agentic infrastructure—with:
🔹 Autonomous coordination
🔹 Real-time policy enforcement
🔹 Cloud/on-prem/air-gapped deployment
🔹 Full lifecycle management (build → test → govern → evolve)

That’s the foundation behind the Agentic OS at aiXplain. A full-stack operating system to help enterprises scale AI agents—without losing control.
Because in the enterprise, AI without architecture is just risk.

♻️ Repost if you’re architecting for agents—not just apps.
 🧠 What’s your agent-to-agent strategy?

Read our blog: Link in comments
""",
    }
]


### 6. Vector Database Setup and Indexing
Creates a vector database using Qdrant to store and retrieve post examples. The database enables semantic search and similarity matching, allowing the agent to find relevant examples based on content themes and generate posts with consistent style.

In [ ]:
# Initialize a vector database (default is Qdrant)
db = IndexFactory.create(name="LinkedIn Post Examples", description="Database for LinkedIn Post examples")
print(db.id) # Your database ID in your dashboard

records = []

# Store data into structured records
for doc in data:
    record = Record(id=doc["id"], value=doc["post"])
    records.append(record)

# Index the records in the database
for i in range(0, len(records), 100):
    db.upsert(records[i: i + 100])

print("Total documents in index:", db.count())


68c9f831eb56a7c09f887845
Total documents in index: 4


### 7. Google Calendar Integration Setup
Configures Google Calendar integration for creating posting reminders and scheduling content publication. This enables users to set up automated reminders for their LinkedIn posting schedule.

In [ ]:
google_cal = ToolFactory.get("<YOUR_GOOGLE_CALENDER_COMPOSIO_INTEGRATION_ID>") # Your Google Cal tool from your dashboard

print("Google Calendar Tool Actions:", [action.code for action in google_cal.actions]) # List all Google Calendar actions

# Select the action that your tool will use
google_cal.action_scope = [action for action in google_cal.actions if action.code == "GOOGLECALENDAR_CREATE_EVENT"]

# Print action code and name
action = [action for action in google_cal.actions if action.code == "GOOGLECALENDAR_CREATE_EVENT"][0]
print(action)

# Print action parameters
google_cal.get_action_inputs(action)

Google Calendar Tool Actions: ['GOOGLECALENDAR_CALENDAR_LIST_INSERT', 'GOOGLECALENDAR_CALENDAR_LIST_UPDATE', 'GOOGLECALENDAR_CALENDARS_DELETE', 'GOOGLECALENDAR_CALENDARS_UPDATE', 'GOOGLECALENDAR_CLEAR_CALENDAR', 'GOOGLECALENDAR_CREATE_EVENT', 'GOOGLECALENDAR_DELETE_EVENT', 'GOOGLECALENDAR_DUPLICATE_CALENDAR', 'GOOGLECALENDAR_EVENTS_INSTANCES', 'GOOGLECALENDAR_EVENTS_LIST', 'GOOGLECALENDAR_EVENTS_MOVE', 'GOOGLECALENDAR_EVENTS_WATCH', 'GOOGLECALENDAR_FIND_EVENT', 'GOOGLECALENDAR_FIND_FREE_SLOTS', 'GOOGLECALENDAR_FREE_BUSY_QUERY', 'GOOGLECALENDAR_GET_CALENDAR', 'GOOGLECALENDAR_GET_CURRENT_DATE_TIME', 'GOOGLECALENDAR_LIST_ACL_RULES', 'GOOGLECALENDAR_LIST_CALENDARS', 'GOOGLECALENDAR_PATCH_CALENDAR', 'GOOGLECALENDAR_PATCH_EVENT', 'GOOGLECALENDAR_QUICK_ADD', 'GOOGLECALENDAR_REMOVE_ATTENDEE', 'GOOGLECALENDAR_SETTINGS_LIST', 'GOOGLECALENDAR_SETTINGS_WATCH', 'GOOGLECALENDAR_SYNC_EVENTS', 'GOOGLECALENDAR_UPDATE_ACL_RULE', 'GOOGLECALENDAR_UPDATE_EVENT']
Action(code=GOOGLECALENDAR_CREATE_EVENT, nam

{'attendees': {'name': 'Attendees',
  'code': 'attendees',
  'value': [],
  'availableOptions': [],
  'datatype': 'array',
  'allowMulti': False,
  'supportsVariables': False,
  'defaultValue': [None],
  'required': False,
  'fixed': False,
  'description': 'List of attendee emails (strings).'},
 'calendar_id': {'name': 'Calendar Id',
  'code': 'calendar_id',
  'value': [],
  'availableOptions': [],
  'datatype': 'string',
  'allowMulti': False,
  'supportsVariables': False,
  'defaultValue': ['primary'],
  'required': False,
  'fixed': False,
  'description': "Target calendar: 'primary' for the user's main calendar, or the calendar's email address."},
 'create_meeting_room': {'name': 'Create Meeting Room',
  'code': 'create_meeting_room',
  'value': [],
  'availableOptions': [],
  'datatype': 'boolean',
  'allowMulti': False,
  'supportsVariables': False,
  'defaultValue': [None],
  'required': False,
  'fixed': False,
  'description': "If true, a Google Meet link is created and added

In [ ]:
# Try an action
google_cal.run("GOOGLECALENDAR_CREATE_EVENT",
{
  "summary": "Post on Social Media",
  "description": "Reminder to post on LinkedIn",
  "event_duration_hour": 0,
  "event_duration_minutes": 30,
  "calendar_id": "primary",
  "timezone": "America/New_York",
  "start_datetime": "2025-09-17T20:00:00"
}
)

ModelResponse(status=SUCCESS, data='{'response_data': {'attendees': [{'email': 'nur.hamdan@aixplain.com', 'organizer': True, 'responseStatus': 'needsAction', 'self': True}], 'created': '2025-09-16T23:53:45.000Z', 'creator': {'email': 'nur.hamdan@aixplain.com', 'self': True}, 'description': 'Reminder to post on LinkedIn', 'end': {'dateTime': '2025-09-17T17:30:00-07:00', 'timeZone': 'America/New_York'}, 'etag': '"3516133651814654"', 'eventType': 'default', 'htmlLink': 'https://www.google.com/calendar/event?eid=ZGd2cTlrNTUwNGhvbnRuYTVudjA5M2Y2c2MgbnVyLmhhbWRhbkBhaXhwbGFpbi5jb20', 'iCalUID': 'dgvq9k5504hontna5nv093f6sc@google.com', 'id': 'dgvq9k5504hontna5nv093f6sc', 'kind': 'calendar#event', 'organizer': {'email': 'nur.hamdan@aixplain.com', 'self': True}, 'reminders': {'useDefault': True}, 'sequence': 0, 'start': {'dateTime': '2025-09-17T17:00:00-07:00', 'timeZone': 'America/New_York'}, 'status': 'confirmed', 'summary': 'Post on Social Media', 'updated': '2025-09-16T23:53:45.907Z'}}', com

### 8. Agent Task Definitions

Defines the individual tasks that comprise the agent workflow. Each task has specific responsibilities, expected outputs, and dependencies that create a logical execution flow from trend discovery to content publication.


In [ ]:
## Agent Tasks
scraped_posts = AgentTask(
    name="Scrape LinkedIn Posts",
    description="Scrape latest trending posts from LinkedIn about the topic mentioned",
    expected_output="Scraped linkedIn post results"
)

post_examples = AgentTask(
    name="Post Examples Index",
    description="Retreive index values to get examples of linkedIn posts",
    expected_output="Index values as examples of how posts should look"
)

generate_post = AgentTask(
    name="Generate LinkedIn Post",
    description="Generate a linkedIn post based on trending topics and use previous posts for tone of the linkedIn post. The posts should highlight aixplain as a platform",
    expected_output="Generated linkedIn post with tonality from previous post examples and trending topics",
    dependencies=[scraped_posts, post_examples]
)

post_on_linkedIn = AgentTask(
    name="Post on LinkedIn",

    description="Publish the generated linkedIn post on linkedIn using the LINKEDIN_CREATE_LINKED_IN_POST tool",
    expected_output="Posted linkedIn post",
    dependencies=[generate_post]
)

google_cal_reminder = AgentTask(
    name = "Google Calender Reminder",
    description="If a user asks to create a reminder for a specified date, create a reminder on google calendar",
    expected_output="Reminder created on google calendar"
)


### 9. Agent Creation and Configuration
Creates the main LinkedIn agent by combining all defined tasks and tools into a cohesive system. The agent can orchestrate the entire workflow from trend analysis to content publication and scheduling.

In [ ]:
linkedIn_agent = AgentFactory.create(
    name="LinkedIn Agent Auto",
    description="An agent that helps you generate LinkedIn Posts",
    tasks=[
        scraped_posts,
        post_examples,
        generate_post,
        post_on_linkedIn,
        google_cal_reminder
    ],
    tools=[
        linkedIn_tool,
        scrape_tool,
        db,
        google_cal
    ]
)

/usr/local/lib/python3.12/dist-packages/aixplain/factories/agent_factory/__init__.py:111: UserWarning: Use `llm` to define the large language model (aixplain.modules.model.llm_model.LLM) to be used as agent. Use `llm_id` to provide the model ID of the large language model to be used as agent. Note: In upcoming releases, `llm` will become a required parameter.
  warnings.warn(
/tmp/ipython-input-2270580513.py:1: DeprecationWarning: The 'tasks' parameter is deprecated and will be removed in a future version. Use 'workflow_tasks' instead.
  linkedIn_agent = AgentFactory.create(


### 10. Agent Execution and Monitoring
Executes the agent with a specific instruction and provides comprehensive monitoring and logging of the execution process. Tracks performance metrics, resource usage, and provides detailed execution analytics.

In [ ]:
response = linkedIn_agent.run("Find trending topics about AI Agents, generate a post about it and publish the new post on linkedIn")
print(response.data.output)

Your LinkedIn post about trending topics on AI Agents has been successfully created and published. Here’s the content of your post:

---

🚀 In the rapidly evolving landscape of AI, the real game-changer is not just automation, but the orchestration of intelligent agents. At aixplain, we are pioneering a platform that empowers businesses to harness the full potential of AI agents, transforming how they engage with their audience and create content. 

💡 While many tools claim to be 'AI agents', true agentic systems go beyond mere prompt following. They reason, act, and adapt in real-time, ensuring that your strategies are not just reactive but proactive. 

🌟 With aixplain, you can streamline your content creation, track performance, and enhance audience interactions, making your brand indispensable in today's fast-paced digital environment. 

Join us in redefining what it means to leverage AI in business. Let's move beyond automation theater and embrace a future where AI truly collaborat

### 11. Detailed Execution Trace Analysis
Provides detailed debugging and analysis capabilities by examining each step of the agent's reasoning process. This includes tool usage, decision-making logic, inputs/outputs, and error handling for comprehensive system observability.

In [ ]:
# Top-level outcome signals from the SDK object
print("Status:", response.status)         # SUCCEEDED / FAILED / IN_PROGRESS (high-level run result)
print("Output:", response.data.output)    # Final answer emitted by the agent
print("Completed:", response.completed)   # True if the run reached :finished/failed
print("Error:", response.error_message)

# Count tool steps across all intermediate steps
tool_step_count = sum(len(step.get("tool_steps") or []) for step in response.data.intermediate_steps or [])
print("Total tool steps:", tool_step_count, "\n")

# ---- Reasoning Trace ----
# The full chain-of-thought
# Each step may include tool calls, inputs/outputs, timings, and any errors captured.
for i, step in enumerate(response.data.intermediate_steps or []):
    print(f"\n--- Step {i+1}: {step.get('agent')} ---")           # Active agent
    print("Thought:", step.get("thought"))                        # Planner note before acting (may be None)
    print("Action:", step.get("action"))                          # Planned action type, e.g., 'call_tool' or 'finish'
    print("Reason:", step.get("reason"))                          # Contains step‑level rationale
    print()


    # Tool call details
    for ts in step.get("tool_steps") or []:
        print("Tool:", ts.get("tool"))                            # Tool name
        print("Input:", ts.get("input"))                          # Input sent to the tool (arguments/payload)
        print("Output:", str(ts.get("output"))[:200], "…")        # Tool output
        print("Reason:", str(ts.get("reason"))[:200], "…")        # Tool output
        print("Error:", ts.get("error"))                          # Any errors during tool execution
        print()

Status: SUCCESS
Output: Your LinkedIn post about trending topics on AI Agents has been successfully created and published. Here’s the content of your post:

---

🚀 In the rapidly evolving landscape of AI, the real game-changer is not just automation, but the orchestration of intelligent agents. At aixplain, we are pioneering a platform that empowers businesses to harness the full potential of AI agents, transforming how they engage with their audience and create content. 

💡 While many tools claim to be 'AI agents', true agentic systems go beyond mere prompt following. They reason, act, and adapt in real-time, ensuring that your strategies are not just reactive but proactive. 

🌟 With aixplain, you can streamline your content creation, track performance, and enhance audience interactions, making your brand indispensable in today's fast-paced digital environment. 

Join us in redefining what it means to leverage AI in business. Let's move beyond automation theater and embrace a future w

### 12. Advanced Agent Execution with Formatting
Demonstrates advanced agent usage with custom output formatting (Markdown) and structured JSON response analysis. This enables better integration with other systems and provides formatted output for different use cases.

In [ ]:
response = linkedIn_agent.run("Find trending topics about AI Agents, generate a post about a topic and create a reminder for Sep 17 morning for me to post about it", output_format="markdown")
print(response.data.output)

```json
{
  "response": {
    "post": "🚀 In the rapidly evolving world of AI, the distinction between true AI agents and mere automation is becoming clearer. At aixplain, we believe that AI should not just follow prompts but should reason, act, and adapt in real-time. \n\nAs we witness the rise of AI agents that promise productivity, it's crucial to ask: Are they truly agentic? Or are they just clever wrappers around traditional automation? 🤔 \n\nWith aixplain, we empower businesses to harness the full potential of AI by providing a platform that integrates advanced algorithms, enabling seamless content creation and audience engagement. \n\nJoin us in redefining what it means to be an AI agent in 2025 and beyond. Let's move past automation theater and embrace a future where AI truly collaborates and enhances our capabilities! 💡 #AI #Aixplain #Innovation #Productivity",
    "reminder": "A reminder has been successfully created for you to post this on September 17th morning."
  }
}
```


In [ ]:
import json

print(json.dumps(response["data"], indent=4))

{
    "input": "{'input': 'Find trending topics about AI Agents, generate a post about a topic and create a reminder for Sep 17 morning for me to post about it', 'chat_history': [], 'outputFormat': 'markdown', 'expectedOutput': 'None'}",
    "output": "```json\n{\n  \"response\": {\n    \"post\": \"\ud83d\ude80 In the rapidly evolving world of AI, the distinction between true AI agents and mere automation is becoming clearer. At aixplain, we believe that AI should not just follow prompts but should reason, act, and adapt in real-time. \\n\\nAs we witness the rise of AI agents that promise productivity, it's crucial to ask: Are they truly agentic? Or are they just clever wrappers around traditional automation? \ud83e\udd14 \\n\\nWith aixplain, we empower businesses to harness the full potential of AI by providing a platform that integrates advanced algorithms, enabling seamless content creation and audience engagement. \\n\\nJoin us in redefining what it means to be an AI agent in 2025 

## Workflow Summary

1. **Trend Discovery**: Agent scrapes LinkedIn for trending AI topics
2. **Style Analysis**: Retrieves similar successful posts from vector database
3. **Content Generation**: Creates new post combining trending topics with established voice
4. **Publication**: Automatically posts to LinkedIn
5. **Scheduling**: Creates calendar reminders for future posting activities

This system provides end-to-end automation for LinkedIn content marketing while maintaining quality and brand consistency.